In [ ]:
!pip install datasets 
!pip install transformers 
!pip install sacrebleu 
!pip install torch 
!pip install sentencepiece 
!pip install transformers[sentencepiece]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

import os
os.environ["WANDB_DISABLED"]="true"

import transformers
print(transformers.__version__)

import datasets
from datasets import load_dataset, load_metric

import random
import pandas as pd

from IPython.display import display, HTML

from transformers import AutoTokenizer

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import MarianMTModel, MarianTokenizer

from sacrebleu.metrics import BLEU

4.15.0


In [ ]:
### same model used for translating texts

pretrained_model = "Helsinki-NLP/opus-mt-en-ro"

In [ ]:
### loading the dataset

raw_datasets = load_dataset("wmt16", "ro-en")
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/38.7M [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/9e0038fe4cc117bd474d2774032cc133e355146ed0a47021b2040ca9db4645c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [ ]:
print(len(raw_datasets["train"]))
print(len(raw_datasets["validation"]))
print(len(raw_datasets["test"]))

610320
1999
1999


In [ ]:
### see how data looks like

raw_datasets["validation"][0]

{'translation': {'en': "Brazil's Former Presidential Chief-of-Staff to Stand Trial",
  'ro': 'Fostul șef al cabinetului prezidențial brazilian este adus în fața instanței'}}

In [ ]:
### We have predictions and labels
### Labels shoud be a list of decoded strings (list of lists)

my_predictions = ["Acesta este un test", "Testul doi"]
my_labels = [["Acesta este un test"], ["Testul doi"]]
metric.compute(predictions=my_predictions, references=my_labels)

In [ ]:
### loading the tokenizer
### we have a pretrained vocabulary => tokenizer: tokenizing + converting the tokens to their corresponding
### IDs from the vocab + generating other inputs that the model requires

### get the tokenizer needed for the model, download vocabulary (used when pretraining this model)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

# vocab_size=59543, model_max_len=512, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'
# not a fast tokenizer (SentencePiece-based tokenizers) => additional methods to map between the original string 
# attention mask (if one sentence is short and has padding, don't pay attention to the padding)

tokenizer(["Get out, please!"], ["Hello, my friend!"])

{'input_ids': [[639, 114, 3, 709, 23, 125, 778, 3, 85, 986, 23, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
# tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"])

In [ ]:
### data preprocessing (data is truncated (max length accepted by the model), but not padded
### padding <= longest length in the batch

prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    # => Membership of Parliament: see Minutes

    targets = [ex[target_lang] for ex in examples["translation"]]
    # => Componenţa Parlamentului: a se vedea procesul-verbal

    model_inputs = tokenizer(inputs, max_length = max_input_length, truncation = True)
    # => 'input_ids' ... 'attention_mask'

    # same tokenizer for the target
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length = max_target_length, truncation = True)
    
    #'input_ids' ... 'attention_mask' ... 'labels'
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

### apply on all the data in the same time
tokenized_datasets = raw_datasets.map(preprocess_function, batched = True)

In [ ]:
### model => from_pretrained

model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model)

Downloading:   0%|          | 0.00/287M [00:00<?, ?B/s]

In [ ]:
###  Seq2SeqTrainingArguments arguments class

batch_size = 16
model_name = pretrained_model.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"/content/drive/MyDrive/MT/model_en_ro",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    weight_decay = 0.01,
    save_total_limit = 2,
    num_train_epochs = 1,
    predict_with_generate = True    
)

# validation is done at the end of each epoch
# no more than 3 saves 

"""Since the Seq2SeqTrainer will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. 
Lastly, we use the predict_with_generate option (to properly generate summaries) and activate mixed
 precision training"""

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


'Since the Seq2SeqTrainer will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. \nLastly, we use the predict_with_generate option (to properly generate summaries) and activate mixed\n precision training'

In [ ]:
### padding inputs to the max length of sentence in batch + padding the labels
### data collators are objects that will form a batch by using a list of dataset elements as input

data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [ ]:
### compute metrics for the predictions
### we should first decode the predictions

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
### training the model

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

trainer.train()

In [ ]:
### import fine tuned model and old model

model_name = '/content/drive/MyDrive/MT/model_en_ro'

finetuned_model = MarianMTModel.from_pretrained(model_name)

tokenizer = MarianTokenizer.from_pretrained(model_name)

old_model = MarianMTModel.from_pretrained(pretrained_model)

In [ ]:
### using the fine tuned model

src_text = raw_datasets["test"][2]["translation"]["en"]

translated = finetuned_model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
translated_sent = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

print(translated_sent)

### using the initial model

translated_old = old_model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
translated_sent_old = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_old]

print(translated_sent_old)


['Procuratura federală a declarat că Jose Dirceu va fi judecat pentru acuzaţiile de corupţie, fraudă şi spălare de bani înaintate la începutul acestei luni.']
['Procuratura federală a declarat că Jose Dirceu va fi judecat în legătură cu corupţia, escrocheria şi acuzaţiile de spălare de bani depuse la începutul acestei luni.']


In [ ]:
### translating raw_datasets.test

def translate_en_to_ro(sent, model, tokenizer):
  translated = model.generate(**tokenizer(sent, return_tensors="pt", padding=True))
  translated_sent = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
  return translated_sent

predicted_targets = []

for example in raw_datasets["test"]["translation"]:
  sent_to_translate = example["en"]
  translated_sent_list = translate_en_to_ro(sent_to_translate, finetuned_model, tokenizer)
  predicted_targets.append(translated_sent_list)

### convert list of lists to list of strings

translated_sent = []
for l in predicted_targets:
  for s in l:
    translated_sent.append(s)

### write them to file

with open('/content/drive/MyDrive/MT/translated_sentences.txt', 'a') as f:
    for item in translated_sent:
        f.write("%s\n" % item)

In [ ]:
### loading translations saved in files => list of strings

with open('/content/drive/MyDrive/MT/translated_sentences.txt') as file:
    translations = file.readlines()

with open('/content/drive/MyDrive/MT/translated_sentences_old.txt') as file:
    translations_old = file.readlines()

### covert target from list of strings to list of lists

targets_lists_of_lists = []
for example in raw_datasets["test"]["translation"]:
  target_sent = example["ro"]
  target_sent_list = []
  target_sent_list.append(target_sent)
  targets_lists_of_lists.append(target_sent_list)

In [ ]:
bleu = BLEU()

result = bleu.corpus_score(translations, targets_lists_of_lists)

result_old = bleu.corpus_score(translations_old, targets_lists_of_lists)

print("Finetuned model: " + str(result))
print()
print("Old model: " + str(result_old))

Finetuned model: BLEU = 48.33 81.8/60.0/44.4/25.0 (BP = 1.000 ratio = 1.000 hyp_len = 11 ref_len = 11)

Old model: BLEU = 33.03 80.0/55.6/37.5/7.1 (BP = 1.000 ratio = 1.000 hyp_len = 10 ref_len = 10)


In [ ]:
print(translations[0])
print(targets_lists_of_lists[0])

In [ ]:
### Examples of old and new translations from Don't patronize me dataset

sent_to_translate = ""

translated_sent_list = translate_en_to_ro(sent_to_translate, finetuned_model, tokenizer)
translated_sent_list_old = translate_en_to_ro(sent_to_translate, old_model, tokenizer)

print(translated_sent_list_old)
print(translated_sent_list)

['Cel mai bun din ambele lumi']
['Cele mai bune din ambele lumi']


In [ ]:
EN: "To bring down high blood sugar levels , insulin needs to be taken . If you are the type who requires insulin during meal time , you will have to take correct doses of insulin in order to lower your blood glucose . Decision needs to be taken on when to inject it and how many times to inject . For this , you have to take the help of a health care professional .
OLD: ['Pentru a reduce valorile mari ale glicemiei, trebuie luată insulina. Dacă sunteţi tipul care necesită insulină în timpul mesei, va trebui să luaţi doze corecte de insulină pentru a vă reduce glicemia. Decizia trebuie luată la momentul injectării şi de câte ori trebuie să vă injectaţi insulina. Pentru aceasta, trebuie să luaţi ajutorul unui profesionist din domeniul sănătăţii.']
NEW: ['Pentru a scădea valorile ridicate ale zahărului din sânge, trebuie luată o doză de insulină. Dacă sunteţi tipul care necesită insulină în timpul mesei, va trebui să utilizaţi doze corecte de insulină pentru a reduce cantitatea de zahăr din sânge. Decizia trebuie luată atunci când trebuie administrată şi de câte ori trebuie să faceţi injecţia. Pentru aceasta, trebuie să luaţi ajutorul unui profesionist din domeniul sănătăţii.']

--- still not so good 
EN: "For those few seconds , humanity is free of its shackles and one glimpses hope amid the hopelessness"
OLD: ['Pentru acele câteva secunde, umanitatea este liberă de cătuşele sale şi de o singură licărire de speranţă în mijlocul disperării.']
NEW: ['Pentru acele câteva secunde, umanitatea este lipsită de cătuşele sale şi de speranţe pe fondul lipsei de speranţă']

EN:  "The Home Office says delays were caused by a computer failure and the arrival of large numbers of vulnerable adults and children "
OLD: ['Home Office spune că întârzierile au fost cauzate de un eşec al calculatorului şi de sosirea unui număr mare de adulţi şi copii vulnerabili']
NEW: ['Oficiul de Interne declară că întârzierile au fost cauzate de un eşec al calculatorului şi de sosirea unui număr mare de adulţi şi copii vulnerabili']

EN: "The neighbouring Baringo South , Baringo North and Mogotio sub-counties have not been spared either , with many people in need of food . "
OLD: ['Învecinate Baringo Sud, Baringo Nord și Mogotio sub-counties nu au fost cruțate nici, cu mulți oameni care au nevoie de alimente.']
NEW: ['Nici sub-judeţele vecine Baringo South, Baringo North şi Mogotio nu au fost cruţate, cu mulţi oameni care au nevoie de hrană.']

EN: "Rescue teams are still present on the island to help the thousands of refugees making the treacherous boat journey to Greece before attempting to head further into Europe"
OLD: ['Echipele de salvare sunt încă prezente pe insulă pentru a ajuta miile de refugiaţi să facă călătoria cu barca trădătoare în Grecia înainte de a încerca să meargă mai departe în Europa.']
NEW: ['Echipele de salvare sunt încă prezente pe insulă pentru a-i ajuta pe miile de refugiaţi să facă călătoria trădătoare cu barca în Grecia înainte de a încerca să se îndrepte mai departe spre Europa.']

EN: "Approve name-change or face ' hopelessness ' "
OLD: ['Aprobați schimbarea numelui sau " lipsa de speranță "']
NEW: ['Aprobaţi schimbarea numelui sau confruntaţi-vă cu lipsa de speranţă']

EN: "Uganda : over 900,000 South Sudanese refugees in need of humanitarian aid"
OLD: ['Uganda: peste 900.000 de refugiați sud-sudanezi care au nevoie de ajutor umanitar']
NEW: ['Uganda: peste 900 000 de refugiaţi din Sudanul de Sud au nevoie de ajutor umanitar']

EN: Examining a series of sectors of the economy , the authors found varying levels of impact on wages resulting from the influx of migrant labour .
OLD: ['Examinând o serie de sectoare ale economiei, autorii au constatat niveluri diferite de impact asupra salariilor rezultate din afluxul de forță de muncă migrantă.']
NEW: ['Analizând o serie de sectoare ale economiei, autorii au descoperit diferite niveluri de impact asupra salariilor cauzate de afluxul de forţă de muncă migrantă.']

EN: Sherk said the public and organizations like the Salvation Army have also been dropping off warm clothes for the men since word got out that they were in need .
OLD: ['Sherk a spus că publicul și organizațiile precum Armata Salvării au fost, de asemenea, dropping off haine calde pentru bărbații din moment ce cuvântul a ieșit că acestea au fost în nevoie.']
NEW: ['Sherk a declarat că publicul şi organizaţiile precum Armata Salvării au lăsat de asemenea haine călduroase pentru bărbaţi de când s-a aflat că au nevoie de ele.']

EN: Talking about family : I met some homeless people in the hospital who were receiving treatment and so on .
OLD: ['Vorbind despre familie: Am întâlnit niște oameni fără adăpost în spital care au fost tratați și așa mai departe.']
NEW: ['Vorbind despre familie: Am întâlnit câţiva oameni fără adăpost în spital care primeau tratament şi aşa mai departe.']

EN:"She decided to be the hope for the hopeless and home for the homeless ."
OLD:['Ea a decis să fie speranța pentru cei fără speranță și acasă pentru cei fără adăpost.']
NEW: ['Ea a decis să fie speranţa celor fără speranţă şi căminul celor fără adăpost.']

EN: "Go home and call it a day"
OLD: ['Du-te acasă şi spune-i o zi.']
NEW: ['Mergeţi acasă şi încheiaţi ziua.']
